In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# === Caminhos ===
DADOS_DIR = Path("Dados/")
RESULTADOS_DIR = Path("Resultados/")
RESULTADOS_DIR.mkdir(exist_ok=True)

In [3]:
# === Função principal ===
def carregar_e_limpar_imdb():
    print("🎬 Carregando dados do IMDB...")

    # 1️⃣ Carrega os arquivos principais do IMDB
    basics = pd.read_csv(
        DADOS_DIR / "title.basics.tsv.gz",
        sep="\t",
        compression="gzip",
        low_memory=False,
        na_values="\\N",
        usecols=[
            "tconst", "titleType", "primaryTitle",
            "originalTitle", "startYear", "runtimeMinutes", "genres"
        ]
    )

    ratings = pd.read_csv(
        DADOS_DIR / "title.ratings.tsv.gz",
        sep="\t",
        compression="gzip",
        low_memory=False,
        na_values="\\N",
        usecols=["tconst", "averageRating", "numVotes"]
    )

    print(f"✅ Dados carregados: {len(basics):,} títulos básicos, {len(ratings):,} avaliações.")

    # 2️⃣ Filtra apenas tipos de interesse (filmes e séries)
    valid_types = ["movie", "tvSeries", "tvMovie"]
    basics = basics[basics["titleType"].isin(valid_types)].copy()

    # 3️⃣ Faz o join com as avaliações (ratings)
    imdb = basics.merge(ratings, on="tconst", how="left")

    # 4️⃣ Limpeza e padronização de tipos
    imdb["startYear"] = pd.to_numeric(imdb["startYear"], errors="coerce").fillna(-1).astype(int)
    imdb["runtimeMinutes"] = pd.to_numeric(imdb["runtimeMinutes"], errors="coerce").fillna(-1)
    imdb["averageRating"] = imdb["averageRating"].fillna(-1)
    imdb["numVotes"] = imdb["numVotes"].fillna(0)
    imdb["genres"] = imdb["genres"].fillna("Unknown")

    # 5️⃣ Remove duplicatas
    imdb = imdb.drop_duplicates(subset=["tconst"], keep="first")

    # 6️⃣ Salva o resultado
    imdb.to_parquet(RESULTADOS_DIR / "IMDB_Limpo.parquet", index=False, engine="fastparquet")

    print(f"🎉 Dados limpos do IMDB salvos em 'Resultados/IMDB_Limpo.parquet'")
    print(f"Total de registros: {len(imdb):,}")

    return imdb

# Executar
imdb_df = carregar_e_limpar_imdb()
imdb_df.head()

🎬 Carregando dados do IMDB...
✅ Dados carregados: 11,987,707 títulos básicos, 1,628,031 avaliações.
🎉 Dados limpos do IMDB salvos em 'Resultados/IMDB_Limpo.parquet'
Total de registros: 1,170,931


,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,1894,45.0,Romance,5.3,229.0
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,1897,100.0,"Documentary,News,Sport",5.3,573.0
2,tt0000502,movie,Bohemios,Bohemios,1905,100.0,Unknown,3.7,23.0
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70.0,"Action,Adventure,Biography",6.0,1030.0
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,1907,90.0,Drama,5.3,34.0
